In [ ]:
include("data/10_ulysses_3.tsp")
n,L,B,K,W_v,w_v,W,coordinates

In [2]:
using JuMP
using CPLEX

In [3]:
function distance(n::Int64,coordinates::Matrix{Float64})
    l=Matrix{Float64}(zeros(n,n))
    for i in 1:n
        for j in 1:n
            l[i,j]=sqrt((coordinates[i,1]-coordinates[j,1])^2+(coordinates[i,2]-coordinates[j,2])^2)
        end 
    end 
    return l
end

distance (generic function with 1 method)

In [4]:
l=distance(n,coordinates)

22×22 Matrix{Float64}:
  0.0        5.88233   5.42148  …   6.60916     6.54622     2.24243
  5.88233    0.0       1.2919      12.024      11.9468      4.10951
  5.42148    1.2919    0.0         11.1724     11.0916      4.06912
  3.34819    4.48743   4.83011      9.85017     9.79409     1.42468
 10.9669    16.7559   16.3883       7.74304     7.81589    12.6968
  8.25804   14.104    13.4684   …   3.3933      3.47242    10.37
  7.31222   13.0906   12.3961       2.17479     2.24876     9.48815
  0.720278   6.06684   5.74943      6.88914     6.83276     2.12059
 11.7082    17.1306   16.2338       5.12957     5.20159    13.9487
  7.93107   13.1973   12.2852       1.38405     1.42562    10.1686
  ⋮                             ⋱               ⋮          
  5.30051   11.1716   10.5983       2.9988      3.01051     7.39024
  6.69123   12.5138   12.1122       4.84298     4.88083     8.49847
  1.41209    6.59335   5.88367  …   5.49627     5.42485     3.49344
  3.94285    2.32261   2.65        10.4

In [5]:
function resolution_statique(n,L,B,K,W_v,w_v,W,coordinates)
    
    l=distance(n,coordinates)
    m=Model(optimizer_with_attributes(CPLEX.Optimizer, "CPX_PARAM_MIPDISPLAY" =>2,"CPX_PARAM_TILIM" => 900))
    @variable(m,x[i=1:n,j=1:n],Bin)
    @variable(m,y[i=1:n,k=1:K],Bin)
    @constraint(m,[i=1:n,j=1:n,k=1:K],x[i,j]+y[i,k]-y[j,k]<=1)
    @constraint(m,[i=1:n,j=1:n,k=1:K],x[i,j]-y[i,k]+y[j,k]<=1)
    @constraint(m,[i=1:n,j=1:n,k=1:K],-x[i,j]+y[i,k]+y[j,k]<=1)
    @constraint(m,[k=1:K],sum(w_v[i]*y[i,k] for i=1:n)<=B)
    @constraint(m,[i=1:n],sum(y[i,k] for k=1:K)==1)
    @objective(m,Min,sum(l[i,j]*x[i,j] for i=1:n,j=1:n))
    return m 
end

resolution_statique (generic function with 1 method)

In [6]:
function resolution_dualite(n,L,B,K,W_v,w_v,W,coordinates)
    l=distance(n,coordinates)
    m=Model(optimizer_with_attributes(CPLEX.Optimizer, "CPX_PARAM_MIPDISPLAY" =>2,"CPX_PARAM_TILIM" => 900))
    @variable(m,alpha>=0)
    @variable(m,beta[i=1:n,j=1:n]>=0)
    @variable(m,x[i=1:n,j=1:n],Bin)
    @variable(m,y[i=1:n,k=1:K],Bin)
    @variable(m,gamma[k=1:K]>=0)
    @variable(m,phi[i=1:n,k=1:K]>=0)



    @constraint(m,[i=1:n,j=1:n,k=1:K],x[i,j]+y[i,k]-y[j,k]<=1)
    @constraint(m,[i=1:n,j=1:n,k=1:K],x[i,j]-y[i,k]+y[j,k]<=1)
    @constraint(m,[i=1:n,j=1:n,k=1:K],-x[i,j]+y[i,k]+y[j,k]<=1)
    @constraint(m,[i=1:n,j=1:n],alpha+beta[i,j]>=(lh[i]+lh[j])*x[i,j])
    @constraint(m,[k=1:K],sum(w_v[i]*y[i,k]+W_v[i]*phi[i,k] for i=1:n)+W*gamma[k]<=B)
    @constraint(m,[i=1:n,k=1:K],gamma[k]+phi[i,k]>=w_v[i]*y[i,k])
    @constraint(m,[i=1:n],sum(y[i,k] for k=1:K)==1)

    @objective(m,Min,sum(l[i,j]*x[i,j]+3*beta[i,j] for i=1:n,j=1:n)+L*alpha)
    return m
end

resolution_dualite (generic function with 1 method)

In [ ]:
include("data/10_ulysses_3.tsp")
ms=resolution_statique(n,L,B,K,W_v,w_v,W,coordinates)
optimize!(ms)

In [ ]:
md=resolution_dualite(n,L,B,K,W_v,w_v,W,coordinates)
optimize!(md)

In [ ]:
data=["data/10_ulysses_3.tsp","data/10_ulysses_6.tsp","data/10_ulysses_9.tsp","data/14_burma_3.tsp","data/14_burma_6.tsp","data/14_burma_9.tsp","data/22_ulysses_3.tsp","data/22_ulysses_6.tsp","data/22_ulysses_9.tsp"]

In [ ]:
for path in data 
    open("results_statique.txt","a") do file 
        include(path)
        ms=resolution_statique(n,L,B,K,W_v,w_v,W,coordinates)
        optimize!(ms)
        println(file,string(n)*"&"*string(K)*"&"*string(objective_value(ms))*"&"*string(solve_time(ms)))
    end
end

In [ ]:
for path in data 
    open("results_dualité.txt","a") do file 
        include(path)
        ms=resolution_dualite(n,L,B,K,W_v,w_v,W,coordinates)
        optimize!(ms)
        println(file,string(n)*"&"*string(K)*"&"*string(objective_value(ms))*"&"*string(solve_time(ms)))
    end
end

In [7]:
path="data/22_ulysses_9.tsp"
open("results_dualité.txt","a") do file 
    include(path)
    ms=resolution_dualite(n,L,B,K,W_v,w_v,W,coordinates)
    optimize!(ms)
    println(file,string(n)*"&"*string(K)*"&"*string(objective_value(ms))*"&"*string(solve_time(ms)))
end

Version identifier: 22.1.0.0 | 2022-03-09 | 1a383f8ce
CPXPARAM_TimeLimit                               900
Tried aggregator 1 time.
MIP Presolve eliminated 1078 rows and 507 columns.
MIP Presolve modified 304 coefficients.
Reduced MIP has 12703 rows, 867 columns, and 38619 nonzeros.
Reduced MIP has 660 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.03 sec. (24.19 ticks)
Found incumbent of value 315.933889 after 0.09 sec. (57.41 ticks)
Probing time = 0.00 sec. (2.31 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 12703 rows, 867 columns, and 38619 nonzeros.
Reduced MIP has 660 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.22 sec. (187.86 ticks)
Probing time = 0.02 sec. (2.14 ticks)
Clique table members: 22.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 0.06 sec. (36.67 ticks)

        Nodes             

In [8]:
path="data/26_eil_3.tsp"
open("results_dualité.txt","a") do file 
    include(path)
    ms=resolution_dualite(n,L,B,K,W_v,w_v,W,coordinates)
    optimize!(ms)
    println(file,string(n)*"&"*string(K)*"&"*string(objective_value(ms))*"&"*string(solve_time(ms)))
end


Cover cuts applied:  4
Implied bound cuts applied:  88
Flow cuts applied:  13
Mixed integer rounding cuts applied:  39
Zero-half cuts applied:  76
Lift and project cuts applied:  1
Gomory fractional cuts applied:  9

Root node processing (before b&c):
  Real time             =    6.31 sec. (3077.04 ticks)
Parallel b&c, 8 threads:
  Real time             =  899.88 sec. (195711.98 ticks)
  Sync time (average)   =  166.45 sec.
  Wait time (average)   =    0.02 sec.
                          ------------
Total (root+branch&cut) =  906.19 sec. (198789.02 ticks)
Version identifier: 22.1.0.0 | 2022-03-09 | 1a383f8ce
CPXPARAM_TimeLimit                               900
Tried aggregator 1 time.
MIP Presolve eliminated 234 rows and 26 columns.
MIP Presolve modified 182 coefficients.
Reduced MIP has 6633 rows, 1486 columns, and 20023 nonzeros.
Reduced MIP has 728 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.26 sec. (9.73 ticks)
Found incumbent of value 103125.298749 after 1.